In [1]:
#Miami, December 27th, 2019
#Renato Kogeyama

# Due bad performance, I am updating the script to more efficient coding

# This script generates a citation ouput with patent|backward citation|cumulated backward citation | year
# The output generates a citation record by year, considering direct and indirect citation


In [2]:
import pandas as pd
import re
import matplotlib.pyplot as plot

import sys
sys.path.append('/home/rkogeyam/scripts/')
from sampler import sampler


In [3]:
%matplotlib inline

In [4]:
fname = r'/home/rkogeyam/PATENT_CITATION/uspatentcitation.tsv'

In [5]:
%%time
# df=pd.read_csv(fname, sep='\t')

sample_size=1000
df=sampler(fname, sample_size, sep='\t')

CPU times: user 5min 36s, sys: 9.63 s, total: 5min 45s
Wall time: 5min 45s


In [6]:
df['patent_id']=df['patent_id'].astype(str)

In [7]:
df['citation_id']=df['citation_id'].astype(str)

In [8]:
df.dtypes

uuid           object
patent_id      object
citation_id    object
date           object
name           object
kind           object
country        object
category       object
sequence        int64
dtype: object

In [9]:
df


,uuid,patent_id,citation_id,date,name,kind,country,category,sequence
0,00h8rtzytugjmhqlmspt10h8g,9439669,5843109,1998-12-01,Mehta et al.,A,US,cited by applicant,218
1,00kapyuimet8xnr5xph1y3o6y,7436128,5410466,1995-04-01,Maehara,A,US,cited by examiner,0
2,03prqufyy03rq3c5xvxpl6ahi,7202696,6608515,2003-08-01,Taheri,B2,US,cited by other,14
3,04iin8oae0g5f6y0ssvqklzna,6451030,5531792,1996-07-01,Huene,A,US,cited by other,19
4,08l1ezzjgi6lcni9x1xbjvv1i,6205503,5715456,1998-02-01,Bennett et al.,NaN,US,NaN,165
5,09hysjndz30105mc6olvrjtx4,8103589,6154207,2000-11-01,Farris et al.,A,US,cited by other,332
6,09z3gbl7j3ifxjq3v07mrqlps,7599342,6601135,2003-07-01,McBrearty et al.,B1,US,cited by other,18
7,0ar3n61z29wig0u3pntgwkvk5,D276620,2926591,1960-03-01,Huffman,NaN,US,NaN,2
8,0b8z9hcgjkm5a9eg2k7tg24dr,5653580,3062499,1962-11-01,Petersen,NaN,US,NaN,1
9,0beq19gs7isdg4e3r2ijigc2r,9038114,6636953,2003-10-01,Yuasa et al.,B2,US,cited by applicant,1


In [10]:
#Data Cleaning
# first, examine the data (patent and citation id)
len(df.citation_id)

615

In [11]:
# drop rows for which column citation_id has a length smaller than 5
indexNames = df[df['citation_id'].map(len) < 5].index
df.drop(indexNames , inplace=True)

In [12]:
df.shape

(615, 9)

In [13]:
# drop rows for which column patent_id has a length smaller than 5
indexNames = df[df['patent_id'].map(len) < 5].index
df.drop(indexNames , inplace=True)

In [14]:
df.shape

(615, 9)

In [15]:
%%time
#stripping non desired characters but keeping the originals for later check
#could merge unwanted information
cleaning_patent=lambda x:re.sub('([a-zA-Z0-9]+)', "", x)
df['citation_id']=df['citation_id'].apply(cleaning_patent)
df['patent_id']=df['patent_id'].apply(cleaning_patent)
# df['citation_id'] = df['citation_id'].str.extract('([a-zA-Z0-9]+)', expand=True)
# df['patent_id'] = df['patent_id'].str.extract('([a-zA-Z0-9]+)', expand=True)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 4.29 ms


In [14]:
df.shape

(615, 9)

In [16]:
#stripping trailing white spaces
df['patent_id'] = df['patent_id'].str.strip()
df['citation_id'] = df['citation_id'].str.strip()

In [14]:
df.shape

(615, 9)

In [17]:
%%time
#to avoid problems when converting object to datetime format
#replacing day or month would not affect the output
df.date.replace({'-00':'-01'}, regex=True, inplace=True)
#replacing the year could be more problematic
#for now, I am just dropping


CPU times: user 1e+03 µs, sys: 0 ns, total: 1e+03 µs
Wall time: 1.23 ms


In [14]:
df.shape

(615, 9)

In [18]:
%%time
df['date']=pd.to_datetime(df.date,format='%Y-%m-%d', errors='coerce') #attention to date format - original data is year-month-day
# df['date_format']=pd.to_datetime(df.date,format='%Y-%m-%d', errors='coerce')

CPU times: user 2 ms, sys: 0 ns, total: 2 ms
Wall time: 935 µs


In [14]:
df.shape

(615, 9)

In [19]:
%%time
#script will take grant year into account
df.sort_values(by='date', ascending=False, inplace=True)

CPU times: user 0 ns, sys: 1e+03 µs, total: 1e+03 µs
Wall time: 875 µs


In [20]:
%%time
#first, i'll run only patents with some kind of citation
#as we have many zero citations in the dataset, this should decrease processing time

# cited_patents=df.groupby(['citation_id']).count().iloc[:, 1] #Series, patent-level
cited_patents=df.groupby(['citation_id']).count().iloc[:, 1] #Series, patent-level

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 2.51 ms


In [21]:
cited_patents

citation_id
    615
Name: patent_id, dtype: int64

In [22]:
%%time
cited_patents.dropna(0, inplace=True) #Series

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 14.5 µs


In [23]:
cited_patents.shape

(1,)

In [24]:
%%time
cited_patents=cited_patents.reset_index() #Dataframe

CPU times: user 1 ms, sys: 0 ns, total: 1 ms
Wall time: 1.23 ms


In [25]:
%%time
cited_patents.head()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 209 µs


,citation_id,patent_id
0,,615


In [26]:
%%time
cited_patents.rename(columns={'patent_id': 'back_citation'}, inplace=True)

CPU times: user 999 µs, sys: 0 ns, total: 999 µs
Wall time: 375 µs


In [27]:
%%time
#this is the trick element
#I match the total of citations received by the citing patent (identified by patent_id)
#The next step I sum all citations received by the citing patents

df=df.merge(cited_patents, how='left', left_on='patent_id', right_on='citation_id')
df.rename(columns={'back_citation': 'parent_patent_back_citation'}, inplace=True)

CPU times: user 4 ms, sys: 1e+03 µs, total: 5 ms
Wall time: 3.9 ms


In [28]:
df.head()

,uuid,patent_id,citation_id_x,date,name,kind,country,category,sequence,citation_id_y,parent_patent_back_citation
0,loryttkavz7n687z1rngp9cb0,,,2015-11-01,Cayo,B2,US,cited by examiner,15,,615
1,h9636hdn1b3pfjz05916ow2iw,,,2015-05-01,Shraim et al.,B2,US,cited by applicant,58,,615
2,jtstcewlet9uols9w5uxw2vx4,,,2015-02-01,Tian,S,US,cited by examiner,37,,615
3,iq0g4ic2kppw4par9oiiuxtsh,,,2014-07-01,Witter,S,US,cited by examiner,9,,615
4,9ozzetvzoemxs7hkxr4l4vw07,,,2014-07-01,Hecker et al.,B2,US,cited by applicant,473,,615


In [29]:
%%time
df.fillna(0, inplace=True)

CPU times: user 2 ms, sys: 0 ns, total: 2 ms
Wall time: 1.89 ms


In [30]:
%%time
#now I collaps df to become a patent level dataframe
#citation_id_x is the original cited patent
patent_level_df=df.groupby('citation_id_x').sum()

CPU times: user 2 ms, sys: 0 ns, total: 2 ms
Wall time: 2.64 ms


In [31]:
# cited_patents.plot.hist()

In [32]:
patent_level_df.head()

,sequence,parent_patent_back_citation
citation_id_x,,
,38595,378225


In [33]:
# lastly, i merge the number of citations to patent_level_df
# df=df.merge(cited_patents, how='left', left_on='patent_id', right_on='citation_id')

In [34]:
# df.describe()

In [35]:
# # cited_patents.iloc[0].citation_id

# # cited_patents.apply(lambda x: df[df.citation_id=='2673233'].patent_id) #so far, it returns a list of patents citing each cited patent

# cited_patents['cumulative']=cited_patents['back_citation']

# # search for citations in df
# # df[df.citation_id=='4025107'].patent_id #series of patents citing '4025107'

# # get cumulative data from cited_patents and sum
# # cited_patents[cited_patents.citation_id=='4025107'].cumulative.sum()

# %%time
# # chained functions
# # cited_patents['cumulative']=cited_patents.apply(lambda x:df[df.citation_id==x].patent_id.apply(lambda y:cited_patents[cited_patents.citation_id==y].cumulative.sum()))

# %%time
# # cumulative=lambda row: cited_patents[df.patent_id==row.citation_id].patent_id.sum()

# # cited_patents['cumulative']=cited_patents.apply(cumulative, axis=1)

# cited_patents.shape

# cited_patents

# # %%time
# # for i in cited_patents.index:
    
# #     citing_patents=df[df.citation_id==i].patent_id  #this is the trick element
# #                                                     #for each cited patent i generate a list of citing patents

# #     cumulative=0 #initialize the citation cumulated variable for each cited patent

# #     cited_patents['cumulative']=(lambda row: df[df.citation_id==row.index].sum().cumulative)
# #     print('Patent:', j, 'Cumulative citations:', cumulative)
        

    
    
# #     #     for j in citing_patents:
# # #         cumulative=+df[df.citation_id==j].sum().cumulative #each cited patent contributes to overall cumulative score

# # #     print(cited_patents.loc[cited_patents['citation_id']==i,'cumulative'])
# # #     cited_patents.at[i,'cumulative']=+cumulative
    

# %%time
# cited_patents.shape

# %%time
# cited_patents.describe()

